In [1]:
import os

import numpy as np
import pandas as pd

from helper import calculate_iqr_range, get_datatype_mapping_for_reduction, reduce_dataset_size

# set global variables
DATA_DIR = "../data/dataset"
max_std = 402198.1853209468
min_std = 0.0

# set maximum column displayable at once
pd.options.display.max_columns = 122

# load the dataset
df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test_data.csv"))
df = reduce_dataset_size(df)
df.info()
df.head()

Reducing dataset size
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float32(51), float64(13), int16(3), int32(1), int64(1), object(16), uint32(1), uint8(36)
memory usage: 86.9+ MB


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,Cash loans,F,N,Y,1,67500.0,227520.0,11065.5,180000.0,Family,Commercial associate,Secondary / secondary special,Married,House / apartment,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,Sales staff,3.0,2,2,WEDNESDAY,12,0,0,0,0,0,0,Business Entity Type 3,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,NaN,block of flats,0.0074,Mixed,No,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,Cash loans,M,Y,Y,1,247500.0,1882372.5,65560.5,1719000.0,Unaccompanied,Working,Higher education,Civil marriage,House / apartment,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,Managers,3.0,2,2,TUESDAY,10,0,0,0,0,1,1,Business Entity Type 3,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,Cash loans,M,Y,Y,1,180000.0,101880.0,10827.0,90000.0,"Spouse, partner",Working,Higher education,Married,House / apartment,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,Laborers,3.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 4,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.16,0.0690,0.4583,0.5,0.1587,0.1614,0.1033,0.0,0.188,0.2017,0.1213,0.9841,0.7909,0.113,0.1611,0.0690,0.4583,0.5,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5,0.1615,0.1642,0.1051,0.0,0.1919,reg oper account,block of flats,0.1789,Panel,No,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,

In [2]:
numerical_columns = list(df.select_dtypes(exclude=["object"]).columns)
print(f"Number of numerical columns: {len(numerical_columns)}\n" )

df[numerical_columns].describe()

Number of numerical columns: 106



,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
count,184506.000000,184506.000000,1.845060e+05,1.845060e+05,184500.000000,1.843390e+05,184506.000000,184506.000000,184506.000000,184506.00000,184506.000000,62873.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184505.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,80432.000000,184093.000000,147850.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.000000,56360.000000,82718.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.00000,56360.000000,82718.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.000000,56360.000000,82718.000000,95558.000000,183867.000000,183867.000000,183867.000000,183867.000000,184505.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.0,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000,184506.000000
mean,278031.658488,0.416463,1.689985e+05,5.992853e+05,27114.455078,5.386926e+05,0.020868,-16036.718307,63838.583271,-4984.89506,-2994.113948,12.049115,0.999995,0.819849,0.199115,0.998043,0.281611,0.056638,2.152706,2.051787,2.031332,12.062963,0.015176,0.051299,0.041175,0.078778,0.230323,0.179257,0.502568,0.514616,0.510894,0.117193,0.088303,0.977795,0.752196,0.044500,0.078558,0.149578,0.225968,0.231827,0.066468,0.100229,0.107146,0.008890,0.028265,0.113888,0.087360,0.977083,0.759342,0.042394,0.074057,0.144987,0.221899,0.227819,0.065124,0.104988,0.10566,0.008094,0.026867,0.117568,0.087798,0.977831,0.755489,0.044459,0.077690,0.149065,0.225543,0

In [3]:
df[numerical_columns]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,1,67500.0,227520.0,11065.5,180000.0,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,3.0,2,2,12,0,0,0,0,0,0,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.0000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,0.0074,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,1,247500.0,1882372.5,65560.5,1719000.0,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,3.0,2,2,10,0,0,0,0,1,1,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,1,180000.0,101880.0,10827.0,90000.0,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,3.0,2,2,14,0,0,0,0,0,0,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.1600,0.0690,0.4583,0.5000,0.1587,0.1614,0.1033,0.0,0.1880,0.2017,0.1213,0.9841,0.7909,0.1130,0.1611,0.0690,0.4583,0.5000,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5000,0.1615,0.1642,0.1051,0.0,0.1919,0.1789,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0,0
3,305311,0,81000.0,405000.0,20677.5,405000.0,0.035792,-12762,-5891,-6430,-4618,NaN,1,1,1,1,1,0,1.0,2,2,18,0,0,0,0,1,1,NaN,0.599924,0.337673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,7.0,1.0,-1671.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
4,414121,0,157500.0,888840.0,29506.5,675000.0,0.018801,-16329,-345,-1413,-4624,NaN,1,1,0,1,0,0,2.0,2,2,9,0,0,0,0,1,1,0.476511,0.312779,0.819318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
def col_info(column_name):
    print(f"{column_name}")
    print(f"Min value: {df[column_name].min()}")
    print(f"Max value: {df[column_name].max()}")
    print(f"Std: {df[column_name].std()}")
    print(f"Min std: {min_std}")
    print(f"Max std:  {max_std}\n")
    
    print(f"Missing value(s): {df[column_name].isna().sum()}")

    print("\n")
    iqr_range, lower_bound, upper_bound = calculate_iqr_range(
                df[column_name],
                scaled_factor=1.7,
                percentile_range=(25, 75)
            )
    print(f"Lower bound: {lower_bound}")
    print(f"Upper bound: {upper_bound}")
    index_of_outliers = df[
                (df[column_name] > upper_bound) | (df[column_name] < lower_bound)].index
    column_index = df[column_name].index
    column_index = column_index.drop(index_of_outliers)
    column_mean_without_outliers = df[column_name][column_index].mean()
    print(f"Number of outlier(s): {len(index_of_outliers)}")    
    mean_of_column = df[column_name].mean()
    print(f"\nMean: {mean_of_column}")
    more_than_mean = list()
    less_than_mean = list()
    for index, value in df[column_name][index_of_outliers].iteritems():
        if value > mean_of_column:
            if value not in more_than_mean:
                more_than_mean.append(value)
        elif value < mean_of_column:
            if value not in less_than_mean:
                less_than_mean.append(value)

            
    less_than_mean.sort()
    more_than_mean.sort()
    print(f"Less than mean: {len(less_than_mean)}")
    print(f"More than mean: {len(more_than_mean)}\n")
    
    unique_values = np.sort(df[column_name].unique())
    print(f"Unique values: {len(unique_values)}")
#     for unique_value in unique_values:
#         print(unique_value, end=", ")

#     print("\n")
#     print(df[column_name].value_counts())
    
    return index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers

# CNT_CHILDREN

In [5]:
column_name = "CNT_CHILDREN"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

CNT_CHILDREN
Min value: 0
Max value: 19
Std: 0.7196143256712833
Min std: 0.0
Max std:  402198.1853209468

Missing value(s): 0


Lower bound: -1.7
Upper bound: 2.7
Number of outlier(s): 2502

Mean: 0.4164634212437536
Less than mean: 0
More than mean: 11

Unique values: 14


In [6]:
print("Outliers:")
df[column_name][index_of_outliers]

Outliers:


78        3
269       3
379       3
406       3
469       3
         ..
184351    3
184353    3
184376    3
184407    3
184466    3
Name: CNT_CHILDREN, Length: 2502, dtype: uint8

In [7]:
df[column_name].replace([3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 3, inplace=True)
df[column_name].value_counts()

0    129082
1     36984
2     15938
3      2502
Name: CNT_CHILDREN, dtype: int64

# AMT_INCOME_TOTAL

In [8]:
column_name = "AMT_INCOME_TOTAL"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_INCOME_TOTAL
Min value: 26100.0
Max value: 117000000.0
Std: 294245.3125
Min std: 0.0
Max std:  402198.1853209468

Missing value(s): 0


Lower bound: -40500.0
Upper bound: 355500.0
Number of outlier(s): 8154

Mean: 168998.5
Less than mean: 0
More than mean: 220

Unique values: 1768


In [9]:
print("Outliers:")
df[column_name][index_of_outliers]

Outliers:


71        630000.0
179       382500.0
195       382500.0
240       360000.0
248       540000.0
            ...   
184349    360000.0
184358    427500.0
184383    450000.0
184387    360000.0
184483    450000.0
Name: AMT_INCOME_TOTAL, Length: 8154, dtype: float32

In [10]:
more_than_mean

[356625.0,
 356850.0,
 357750.0,
 358200.0,
 359100.0,
 360000.0,
 360450.0,
 360508.5,
 362250.0,
 362911.5,
 364500.0,
 364891.5,
 365850.0,
 366750.0,
 369000.0,
 371250.0,
 371392.1875,
 371925.0,
 373500.0,
 373950.0,
 374220.0,
 375750.0,
 376875.0,
 377145.0,
 378000.0,
 378900.0,
 380700.0,
 381150.0,
 381375.0,
 382500.0,
 383400.0,
 385200.0,
 386473.5,
 387000.0,
 388350.0,
 389250.0,
 391500.0,
 391972.5,
 392625.0,
 392850.0,
 394353.0,
 395100.0,
 396000.0,
 396922.5,
 400500.0,
 402750.0,
 403200.0,
 403650.0,
 405000.0,
 409500.0,
 410400.0,
 410850.0,
 411300.0,
 411750.0,
 414000.0,
 415813.5,
 416029.5,
 416322.0,
 418500.0,
 422100.0,
 423000.0,
 424350.0,
 426019.5,
 427050.0,
 427500.0,
 430650.0,
 432000.0,
 432981.0,
 433350.0,
 433800.0,
 434250.0,
 436495.5,
 436500.0,
 438750.0,
 440100.0,
 441000.0,
 443250.0,
 443700.0,
 445500.0,
 450000.0,
 450225.0,
 452250.0,
 454500.0,
 458550.0,
 459000.0,
 460417.5,
 463500.0,
 466956.0,
 468000.0,
 469800.0,
 472500

In [11]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] >= 9000000)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184506
Rows after: 184502


In [12]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 8154
After deleting: 8150


In [13]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 356625.0
Index of lowest outlier value in unique values: 1548
Outliers will be trimmed to: 355500.0


In [14]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

71        355500.0
179       355500.0
195       355500.0
240       355500.0
248       355500.0
            ...   
184349    355500.0
184358    355500.0
184383    355500.0
184387    355500.0
184483    355500.0
Name: AMT_INCOME_TOTAL, Length: 8150, dtype: float32

# AMT_CREDIT

In [15]:
column_name = "AMT_CREDIT"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_CREDIT
Min value: 45000.0
Max value: 4050000.0
Std: 402193.53125
Min std: 0.0
Max std:  402198.1853209468

Missing value(s): 0


Lower bound: -645705.0
Upper bound: 1724355.0
Number of outlier(s): 3181

Mean: 599276.3125
Less than mean: 0
More than mean: 387

Unique values: 4854


In [16]:
more_than_mean

[1724688.0,
 1725120.0,
 1726866.0,
 1728000.0,
 1729615.5,
 1730511.0,
 1730655.0,
 1731901.5,
 1732500.0,
 1734543.0,
 1735474.5,
 1736937.0,
 1737000.0,
 1739470.5,
 1740294.0,
 1741077.0,
 1741293.0,
 1741500.0,
 1741972.5,
 1743399.0,
 1744398.0,
 1745113.5,
 1746000.0,
 1746288.0,
 1746684.0,
 1747008.0,
 1748835.0,
 1749325.5,
 1750500.0,
 1752039.0,
 1755000.0,
 1756714.5,
 1757074.5,
 1759180.5,
 1759500.0,
 1759576.5,
 1762110.0,
 1764000.0,
 1764108.0,
 1767145.5,
 1768248.0,
 1768500.0,
 1769035.5,
 1769220.0,
 1772181.0,
 1772352.0,
 1772406.0,
 1773000.0,
 1773963.0,
 1774039.5,
 1777212.0,
 1777500.0,
 1778890.5,
 1779030.0,
 1782000.0,
 1782247.5,
 1783818.0,
 1786500.0,
 1787283.0,
 1788502.5,
 1788750.0,
 1791000.0,
 1792318.5,
 1793677.5,
 1795203.0,
 1795500.0,
 1795972.5,
 1797354.0,
 1798141.5,
 1798416.0,
 1798605.0,
 1800000.0,
 1802385.0,
 1803532.5,
 1804500.0,
 1805922.0,
 1805985.0,
 1807420.5,
 1808842.5,
 1809000.0,
 1812456.0,
 1813500.0,
 1817428.5,
 181

In [17]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 1724688.0
Index of lowest outlier value in unique values: 4467
Outliers will be trimmed to: 1724220.0


In [18]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

1         1724220.0
68        1724220.0
71        1724220.0
116       1724220.0
152       1724220.0
            ...    
184126    1724220.0
184131    1724220.0
184261    1724220.0
184387    1724220.0
184495    1724220.0
Name: AMT_CREDIT, Length: 3181, dtype: float32

# AMT_ANNUITY

In [19]:
column_name = "AMT_ANNUITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_ANNUITY
Min value: 1615.5
Max value: 258025.5
Std: 14464.37109375
Min std: 0.0
Max std:  402198.1853209468

Missing value(s): 6


Lower bound: -14113.349999999999
Upper bound: 65264.85
Number of outlier(s): 3191

Mean: 27113.099609375
Less than mean: 0
More than mean: 1201

Unique values: 12411


In [20]:
test_df[column_name].isna().sum()

6

In [21]:
test_df[column_name]

0         25317.0
1         21069.0
2         36553.5
3         20682.0
4         31018.5
           ...   
123000    32602.5
123001    42790.5
123002    26851.5
123003    31972.5
123004    36211.5
Name: AMT_ANNUITY, Length: 123005, dtype: float64

In [22]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] >= 258025.5)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184502
Rows after: 184501


In [23]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 3191
After deleting: 3190


In [24]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 65272.5
Index of lowest outlier value in unique values: 11209
Outliers will be trimmed to: 65227.5


In [25]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

1         65227.5
15        65227.5
25        65227.5
37        65227.5
304       65227.5
           ...   
184126    65227.5
184131    65227.5
184134    65227.5
184215    65227.5
184261    65227.5
Name: AMT_ANNUITY, Length: 3190, dtype: float32

In [26]:
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
68774,157917,Cash loans,F,N,N,0,94500.0,450000.0,NaN,450000.0,Unaccompanied,Working,Lower secondary,Civil marriage,House / apartment,0.035792,-9027,-1270,-3640,-741,NaN,1,1,1,1,0,0,Laborers,2.0,2,2,MONDAY,20,0,0,0,0,0,0,Business Entity Type 1,NaN,0.727274,0.468660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-706.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0
87443,163757,Cash loans,F,N,N,0,162000.0,296280.0,NaN,225000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.035792,-11329,-2040,-3195,-3069,NaN,1,1,0,1,0,0,Core staff,2.0,2,2,FRIDAY,13,0,0,0,1,1,1,Government,0.336803,0.566316,0.220095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2841.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0,0
114531,379997,Cash loans,F,N,N,0,315000.0,1483231.5,NaN,1354500.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.072508,-15072,-152,-7576,-4389,NaN,1,1,1,1,1,0,Accountants,2.0,1,1,WEDNESDAY,15,0,1,1,0,1,1,Self-employed,0.593394,0.267269,NaN,0.3691,0.1891,0.9806,0.7348,0.2590,0.52,0.2241,0.5417,0.5833,0.0000,0.3009,0.3756,0.0000,0.2002,0.2216,0.1435,0.9796,0.7321,0.1590,0.3222,0.1379,0.5417,0.5833,0.0000,0.1938,0.2268,0.0000,0.0720,0.3726,0.1891,0.9806,0.7383,0.2607,0.52,0.2241,0.5417,0.5833,0.0000,0.3061,0.3824,0.0000,0.2044,reg oper spec account,block of flats,0.1860,"Stone, brick",No,0.0,0.0,0.0,0.0,-504.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
147768,187985

In [27]:
df[column_name].fillna(column_mean_without_outliers, inplace=True)
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
